# Get Area-Elevation Curve

In [1]:
import ee
ee.Initialize(project='tmospp')

In [2]:
# from rat.ee_utils.ee_aec_file_creator import aec_file_creator
import geopandas as gpd
from pathlib import Path
import hvplot.pandas
import pandas as pd
import holoviews as hv
import geoviews as gv
import numpy as np

hv.extension('bokeh')

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f6d46ddc500>>
Traceback (most recent call last):
  File "/tiger1/pdas47/tmsosPP/.env/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


### select the reservoir

In [28]:
start_date = '2023-07-21'
end_date = '2024-10-30'
RESERVOIR = '1078'

ALG_TYPE = 'swot'
DATA_DIR = Path('../data')

In [29]:
# read the bounding box of the study area
val_pts = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-pts.geojson'))
val_polys = gpd.read_file(Path('../data/validation-locations/2023-24-insitu-poly.geojson'))

selected_reservoirs = val_pts['tmsos_id'].tolist()
res_names = val_pts[['tmsos_id', 'name']].set_index('tmsos_id').to_dict()['name']

RESERVOIR_NAME = res_names[RESERVOIR]
print(f'{RESERVOIR}: {RESERVOIR_NAME}')

val_res_pt = val_pts.loc[val_pts['tmsos_id'].isin(selected_reservoirs)]
val_res_poly = val_polys.loc[val_polys['tmsos_id'].isin(selected_reservoirs)]

nominal_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_SKM'].values[0]
nominal_area_poly = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_POLY'].values[0]
max_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MAX'].values[0]
max_area = np.nan if max_area == -99 else max_area
min_area = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_MIN'].values[0]
min_area = 0 if min_area == -99 else min_area
area_rep = val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['AREA_REP'].values[0]
dam_height = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DAM_HGT_M'].values[0])
elev_msl = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['ELEV_MASL'].values[0])
depth = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['DEPTH_M'].values[0])
capacity = float(val_res_poly[val_res_poly['tmsos_id'] == RESERVOIR]['CAP_MCM'].values[0])

global_map = (
    val_res_pt.hvplot(
        geo=True, tiles='OSM'
    ) * val_res_pt[val_res_pt['tmsos_id'] == RESERVOIR].hvplot(
        geo=True, color='red', size=100, 
    )
).opts(
    title=f"Locations of validation reservoirs. {RESERVOIR_NAME}, highlighted in red"
)

global_map

1078: Lake Mead, US


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Longitude,Latitude]
   .Points.II :Points   [Longitude,Latitude]

In [30]:
val_polys['db'] = val_polys['db'].where(val_polys['rise_id'].isna(), 'rise')

## Storage Calculation

In [31]:
# what is the reported capacity?
capacity_hv = hv.HLine(capacity).opts(color='red', ylim=(0, capacity + capacity*0.1), ylabel='capacity (Mil. m3)')
capacity_hv

:HLine   [x,y]

In [32]:
aec_dir = Path('../data/aec/aev')

aec_df = pd.read_csv(aec_dir / f'{RESERVOIR}.csv', comment='#')

# Calculate storage using satellite AEV

In [33]:
aec_df.hvplot.scatter(
    x='CumArea', y='Elevation', 
    # by='obs_or_extrapolated'
).opts(
    height=400, width=500,
    title=f'{RESERVOIR}: {RESERVOIR_NAME}\nAEC',
    xlabel='Area (km2)', ylabel='Elevation (m a.s.l)'
)

:Scatter   [CumArea]   (Elevation)

In [34]:
aec_df.hvplot(x='CumArea', y='Elevation').opts(height=400, width=500, title=f'{RESERVOIR}: {RESERVOIR_NAME}  [A-E]') + aec_df.hvplot(x='Elevation', y='Storage (mil. m3)', title=f'{RESERVOIR}: {RESERVOIR_NAME}  [S-E]').opts(height=400, width=500) * capacity_hv

:Layout
   .Curve.I   :Curve   [CumArea]   (Elevation)
   .Overlay.I :Overlay
      .Curve.I :Curve   [Elevation]   (Storage (mil. m3))
      .HLine.I :HLine   [x,y]

In [35]:
# read in-situ data
import xarray as xr

def get_insitu_df(
        tmsos_id,
        val_polys,
        aec,
        deltares_insitu_dir=Path('../data/insitu/deltares'),
        rid_insitu_dir=Path('../data/insitu/rid'),
        resops_insitu_dir=Path('../data/insitu/resopsus'),
        rise_insitu_dir=Path('../data/insitu/rise')
    ):
    row = val_polys[val_polys['tmsos_id']==tmsos_id]
    db = row['db'].values

    print(db)

    insitu_df = None
    rise_id = None
    if db == 'rise':
        rise_id = row['rise_id'].item()
        print(rise_id)
        # if rise_id exists, prefer using that database since it has latest data        
        storage_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Storage.csv'

        insitu_df = pd.read_csv(storage_fn, parse_dates=['Datetime (UTC)'])
        assert insitu_df['Units'].iloc[0] == 'af'  # make sure unit is acre-feet
        columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
        insitu_df = insitu_df.drop(columns=[col for col in columns_to_drop if col in insitu_df.columns])
        insitu_df['Result'] = insitu_df['Result'] * 1233.48183   # convert from acre-feet to m3
        insitu_df['date'] = pd.to_datetime(insitu_df['Datetime (UTC)'].dt.date)
        insitu_df = insitu_df.rename({'Result': 'observed storage [m3]'}, axis=1)
        insitu_df = insitu_df.drop('Datetime (UTC)', axis=1)

        insitu_df['observed storage [Mm3]'] = insitu_df['observed storage [m3]'] * 1e-6  # convert to million m3
        insitu_df['db'] = 'rise'

        area_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Area.csv'
        if area_fn.exists():
            insitu_area_df = pd.read_csv(area_fn, parse_dates=['Datetime (UTC)'])
            assert insitu_area_df['Units'].iloc[0] == 'acres'  # make sure unit is acre
            columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
            insitu_area_df = insitu_area_df.drop(columns=[col for col in columns_to_drop if col in insitu_df.columns])
            insitu_area_df['Result'] = insitu_area_df['Result'] * 0.0040468564   # convert from acre-feet to km2
            insitu_area_df['date'] = pd.to_datetime(insitu_area_df['Datetime (UTC)'].dt.date)
            insitu_area_df = insitu_area_df.rename({'Result': 'observed area [km2]'}, axis=1)
            insitu_area_df = insitu_area_df.drop('Datetime (UTC)', axis=1)
            insitu_df['observed area [km2]'] = insitu_area_df['observed area [km2]']
            insitu_df['calculated area [km2]'] = np.nan
        else:
            insitu_df['observed area [km2]'] = np.nan
            insitu_df['calculated area [km2]'] = np.interp(insitu_df['observed storage [Mm3]'], aec['Storage (mil. m3)'], aec['CumArea'])

        elevation_fn = rise_insitu_dir / rise_id / 'Lake_Reservoir_Elevation.csv'
        if elevation_fn.exists():
            insitu_elevation_df = pd.read_csv(elevation_fn, parse_dates=['Datetime (UTC)'])
            assert insitu_elevation_df['Units'].iloc[0] == 'ft'  # make sure unit is acre
            columns_to_drop = ['Unnamed: 0', 'Location', 'Parameter', 'Units', 'Timestep', 'Aggregation', 'timeStep', 'resultType']
            insitu_elevation_df = insitu_elevation_df.drop(columns=[col for col in columns_to_drop if col in insitu_df.columns])
            insitu_elevation_df['Result'] = insitu_elevation_df['Result'] * 0.3048   # convert from feet to m
            insitu_elevation_df['date'] = pd.to_datetime(insitu_elevation_df['Datetime (UTC)'].dt.date)
            insitu_elevation_df = insitu_elevation_df.rename({'Result': 'observed wse [m]'}, axis=1)
            insitu_elevation_df = insitu_elevation_df.drop('Datetime (UTC)', axis=1)
            insitu_df['observed wse [m]'] = insitu_elevation_df['observed wse [m]']
            insitu_df['calculated wse [m]'] = np.nan
        else:
            insitu_df['observed wse [m]'] = np.nan
            insitu_df['calculated wse [m]'] = np.interp(insitu_df['observed storage [Mm3]'], aec['Storage (mil. m3)'], aec['Elevation'])

        insitu_df['calculated storage [m3]'] = np.nan
        insitu_df['calculated storage [Mm3]'] = np.nan

    elif db == 'deltares':
        deltares_id = row['deltares_id'].values[0]
        
        fn = deltares_insitu_dir / f'{int(deltares_id):07}.csv'
        
        insitu_df = pd.read_csv(fn, parse_dates=['time'])
        insitu_df.sort_values('time', inplace=True)
        insitu_df['date'] = pd.to_datetime(insitu_df['time'].dt.date)
        insitu_df['observed area [km2]'] = insitu_df['area'] * 1e-6
        insitu_df['observed wse [m]'] = np.nan
        insitu_df['observed storage [Mm3]'] = np.nan
        insitu_df['db'] = 'deltares'
        insitu_df = insitu_df[['date', 'observed area [km2]', 'observed wse [m]', 'observed storage [Mm3]', 'db']]
        
        # calculate other columns
        insitu_df['observed storage [m3]'] = insitu_df['observed storage [Mm3]'] * 1e6

        calculated_wse = np.interp(insitu_df['observed area [km2]'], aec['CumArea'], aec['Elevation'])
        insitu_df['calculated wse [m]'] = calculated_wse
        
        calculated_storage = np.interp(insitu_df['observed area [km2]'], aec['CumArea'], aec['Storage'])
        insitu_df['calculated storage [m3]'] = calculated_storage
        insitu_df['calculated storage [Mm3]'] = calculated_storage * 1e-6

    elif db == 'rid':
        rid_filename = row['rid_filename'].values[0]
        
        fn = rid_insitu_dir / rid_filename
        insitu_df = pd.read_csv(fn, parse_dates=['date'])
        insitu_df['observed area [km2]'] = np.nan
        insitu_df['observed wse [m]'] = insitu_df['water_level (m)']
        insitu_df['observed storage [Mm3]'] = insitu_df['storage (mil. m3)']
        insitu_df['db'] = 'rid'
        insitu_df = insitu_df[['date', 'observed area [km2]', 'observed wse [m]', 'observed storage [Mm3]', 'db']]

        # calculate other columns
        insitu_df['observed storage [m3]'] = insitu_df['observed storage [Mm3]'] * 1e6

        insitu_df['calculated storage [m3]'] = np.nan
        insitu_df['calculated storage [Mm3]'] = np.nan

        insitu_df['calculated wse [m]'] = np.nan
        insitu_df['calculated area [km2]'] = np.interp(insitu_df['observed storage [m3]'], aec['Storage'], aec['CumArea'])

    elif db == 'resops':
        resops_id = int(row['resops_id'].values[0])
        fn = resops_insitu_dir / f'ResOpsUS_{resops_id}.csv'

        insitu_df = pd.read_csv(fn, parse_dates=['date'])
        insitu_df['observed area [km2]'] = np.nan
        insitu_df['observed wse [m]'] = insitu_df['elevation']
        insitu_df['observed storage [Mm3]'] = insitu_df['storage']
        insitu_df['db'] = 'resops'
        insitu_df = insitu_df[['date', 'observed area [km2]', 'observed wse [m]', 'observed storage [Mm3]', 'db']]

        # calculate other columns
        insitu_df['observed storage [m3]'] = insitu_df['observed storage [Mm3]'] * 1e6

        calculated_storage = np.interp(insitu_df['observed wse [m]'], aec['Elevation'], aec['Storage'])
        insitu_df['calculated storage [m3]'] = calculated_storage
        insitu_df['calculated storage [Mm3]'] = calculated_storage * 1e-6

        insitu_df['calculated wse [m]'] = np.nan
        insitu_df['calculated area [km2]'] = np.interp(insitu_df['observed wse [m]'], aec['Elevation'], aec['CumArea'])
    
    # some days there are repeated data. take the average in those cases.
    insitu_df = insitu_df.groupby('date').agg({
        "db": lambda x: x.iloc[0],
        "observed area [km2]": 'mean',
        "calculated area [km2]": 'mean',
        "observed wse [m]": 'mean',
        "calculated wse [m]": 'mean',
        "observed storage [m3]": 'mean',
        "calculated storage [m3]": 'mean',
        "observed storage [Mm3]": 'mean',
        "calculated storage [Mm3]": 'mean',
    }).reset_index()

    dt = insitu_df['date'].diff().dt.days

    if 'observed storage [m3]' in insitu_df.columns:
        insitu_df['observed storage_change [m3]'] = insitu_df['observed storage [m3]'].diff()
        insitu_df['observed storage_change [Mm3]'] = insitu_df['observed storage [Mm3]'].diff()
        insitu_df['observed storage_change_rate [m3]'] = insitu_df['observed storage_change [m3]'] / dt
        insitu_df['observed storage_change_rate [Mm3]'] = insitu_df['observed storage_change [Mm3]'] / dt
    else:
        insitu_df['observed storage_change [m3]'] = np.nan
        insitu_df['observed storage_change [Mm3]'] = np.nan
        insitu_df['observed storage_change_rate [m3]'] = np.nan
        insitu_df['observed storage_change_rate [Mm3]'] = np.nan

    insitu_df['calculated storage_change [m3]'] = insitu_df['calculated storage [m3]'].diff()
    insitu_df['calculated storage_change [Mm3]'] = insitu_df['calculated storage [Mm3]'].diff()
    insitu_df['calculated storage_change_rate [m3]'] = insitu_df['calculated storage_change [m3]'] / dt
    insitu_df['calculated storage_change_rate [Mm3]'] = insitu_df['calculated storage_change [Mm3]'] / dt

    return insitu_df

def insitu_df_determine_caluclated_or_observed(insitu_df, variable, subset_idx=slice(None)):
    """
    Determines the appropriate storage variable name based on the database type and availability of data.
    
    Parameters:
        insitu_df (pd.DataFrame): DataFrame containing the in-situ data with a 'db' column indicating the database type.
        variable (str): The variable name for which the storage is being determined.
        subset_idx (slice, optional): A slice object to subset the DataFrame. Defaults to slice(None, None).
    
    Returns:
        str: The name of the storage variable to be used.

    """
    if variable == 'storage':
        unit = '[Mm3]'
    elif variable == 'wse':
        unit = '[m]'
    elif variable == 'area':
        unit = '[km2]'
    
    var = f'calculated {variable} {unit}'
    if not insitu_df.loc[subset_idx, var.replace("calculated", "observed")].isna().all():
        # if observed storage is not available, use calculated storage
        var = var.replace("calculated", "observed")

    return var

print(RESERVOIR)
insitu_df = get_insitu_df(RESERVOIR, val_polys, aec_df)
insitu_df
# # storage_var = insitu_df_determine_caluclated_or_observed(insitu_df, 'wse')
# # storage_var
# val_polys[val_polys['tmsos_id'] == RESERVOIR]

1078
['rise']
4370


,date,db,observed area [km2],calculated area [km2],observed wse [m],calculated wse [m],observed storage [m3],calculated storage [m3],observed storage [Mm3],calculated storage [Mm3],observed storage_change [m3],observed storage_change [Mm3],observed storage_change_rate [m3],observed storage_change_rate [Mm3],calculated storage_change [m3],calculated storage_change [Mm3],calculated storage_change_rate [m3],calculated storage_change_rate [Mm3]
0,2013-01-01,rise,NaN,603.718751,341.519256,NaN,1.683357e+10,NaN,16833.573230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-02,rise,NaN,603.701372,341.516208,NaN,1.683234e+10,NaN,16832.339749,NaN,-1.233482e+06,-1.233482,-1.233482e+06,-1.233482,NaN,NaN,NaN,NaN
2,2013-01-03,rise,NaN,603.650974,341.507064,NaN,1.682876e+10,NaN,16828.762651,NaN,-3.577097e+06,-3.577097,-3.577097e+06,-3.577097,NaN,NaN,NaN,NaN
3,2013-01-04,rise,NaN,603.668353,341.510112,NaN,1.683000e+10,NaN,16829.996133,NaN,1.233482e+06,1.233482,1.233482e+06,1.233482,NaN,NaN,NaN,NaN
4,2013-01-05,rise,NaN,603.819499,341.537544,NaN,1.684073e+10,NaN,16840.727425,NaN,1.073129e+07,10.731292,1.073129e+07,10.731292,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4301,2024-10-11,rise,NaN,505.177986,324.206616,NaN,1.073603e+10,NaN,10736.028491,NaN,-2.849343e+06,-2.849343,-2.849343e+06,-2.849343,NaN,NaN,NaN,NaN
4302,2024-10-12,rise,NaN,505.285392,324.224904,NaN,1.074174e+10,NaN,10741.739512,NaN,5.711021e+06,5.711021,5.711021e+06,5.711021,NaN,NaN,NaN,NaN
4303,2024-10-13,rise,NaN,505.285392,324.224904,NaN,1.074174e+10,NaN,10741.739512,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,NaN
4304,2024-10-14,rise,NaN,505.177986,324.206616,NaN,1.073603e+10,NaN,10736.028491,NaN,-5.711021e+06,-5.711021,-5.711021e+06,-5.711021,NaN,NaN,NaN,NaN


In [36]:
import xarray as xr
import hvplot.xarray
import numpy as np

tmsos_dir = Path('../data/tmsos')
tmsos_fp = tmsos_dir / f'{RESERVOIR}.csv'

karin_elevation_dir = Path('../data/elevation/swot_karin')
karin_elevation_fp = karin_elevation_dir / f'{RESERVOIR}.csv'

swot_nadir_dir = Path('../data/elevation/swot_nadir')
swot_nadir_fp = swot_nadir_dir / f'{RESERVOIR}.csv'

data = {
    'elevation': [],
    'date': [],
    'area': []
}
if ALG_TYPE == 'tmsos':
    tmsos_df = pd.read_csv(tmsos_fp, parse_dates=['date'])
    data['area'] = tmsos_df['area']
    data['date'] = tmsos_df['date'] # add area and date from the tmsos data
    data['elevation'] = [np.nan] * len(tmsos_df['area'])
elif ALG_TYPE == 'swot':
    swot_karin_df = pd.read_csv(karin_elevation_fp, parse_dates=['time'])
    data['elevation'] = swot_karin_df['elevation']
    data['date'] = swot_karin_df['time']
    if swot_nadir_fp.exists():
        swot_poseidon_df = pd.read_csv(swot_nadir_fp, parse_dates=['time'])
        data['elevation'] = pd.concat([data['elevation'], swot_poseidon_df['elevation']])
        data['date'] = pd.concat([data['date'], swot_poseidon_df['time']])
    data['area'] = [np.nan] * len(data['elevation'])
elif ALG_TYPE == 'insitu':
    insitu_df = get_insitu_df(RESERVOIR, val_polys, aec_df)
    elevation_var = insitu_df_determine_caluclated_or_observed(insitu_df, "wse")
    area_var = insitu_df_determine_caluclated_or_observed(insitu_df, "area")
    storage_var = insitu_df_determine_caluclated_or_observed(insitu_df, "storage")

    print(elevation_var, area_var, storage_var)
    
    data['elevation'] = insitu_df[elevation_var]
    data['area'] = insitu_df[area_var]
    data['storage'] = insitu_df[storage_var] * 1e6 # mil. m3 to m3
    data['date'] = insitu_df['date']

reservoir_dynamics = pd.DataFrame(data).set_index('date')
reservoir_dynamics = reservoir_dynamics.groupby('date').mean('elevation') # take a mean when multiple observations for same date is available
reservoir_dynamics = reservoir_dynamics.sort_index()

reservoir_dynamics = reservoir_dynamics.to_xarray()

# compute elevation values using aec nad set attributes
if ALG_TYPE == 'tmsos':
    reservoir_dynamics['area'].attrs['alg_type'] = 'tmsos'
    reservoir_dynamics['area'].attrs['obs_imp'] = 'obs'
    reservoir_dynamics['area'].attrs['unit'] = 'km^2'
    elevation = np.interp(reservoir_dynamics['area'], aec_df['CumArea'], aec_df['Elevation'])
    elevation_da = xr.DataArray(data=elevation, coords=reservoir_dynamics.coords)
    storage = np.interp(reservoir_dynamics['area'], aec_df['CumArea'], aec_df['Storage'])
    storage_da = xr.DataArray(data=storage, coords=reservoir_dynamics.coords)
    reservoir_dynamics = reservoir_dynamics.assign(elevation = elevation_da)
    reservoir_dynamics = reservoir_dynamics.assign(storage = storage_da)
elif ALG_TYPE == 'swot':
    reservoir_dynamics['elevation'].attrs['alg_type'] = 'swot'
    reservoir_dynamics['elevation'].attrs['obs_imp'] = 'obs'
    reservoir_dynamics['elevation'].attrs['unit'] = 'm'
    
    area = np.interp(reservoir_dynamics['elevation'], aec_df['Elevation'], aec_df['CumArea'])
    area_da = xr.DataArray(data=area, coords=reservoir_dynamics.coords)
    
    storage = np.interp(reservoir_dynamics['elevation'], aec_df['Elevation'], aec_df['Storage'])
    storage_da = xr.DataArray(data=storage, coords=reservoir_dynamics.coords)
    
    reservoir_dynamics = reservoir_dynamics.assign(area = area_da)
    reservoir_dynamics = reservoir_dynamics.assign(storage = storage_da)
elif ALG_TYPE == 'insitu':
    insitu_df = get_insitu_df(RESERVOIR, val_polys, aec_df)
    elevation_var = insitu_df_determine_caluclated_or_observed(insitu_df, "wse")
    area_var = insitu_df_determine_caluclated_or_observed(insitu_df, "area")
    storage_var = insitu_df_determine_caluclated_or_observed(insitu_df, "storage")

    reservoir_dynamics['elevation'].attrs['alg_type'] = 'insitu'
    reservoir_dynamics['elevation'].attrs['obs_imp'] = 'obs' if 'observed' in elevation_var else 'imp'
    reservoir_dynamics['elevation'].attrs['unit'] = 'm'
    
    reservoir_dynamics['area'].attrs['alg_type'] = 'insitu'
    reservoir_dynamics['area'].attrs['obs_imp'] = 'obs' if 'observed' in area_var else 'imp'
    reservoir_dynamics['area'].attrs['unit'] = 'km^2'
    
    reservoir_dynamics['storage'].attrs['alg_type'] = 'insitu'
    reservoir_dynamics['storage'].attrs['obs_imp'] = 'obs' if 'observed' in storage_var else 'imp'
    reservoir_dynamics['storage'].attrs['unit'] = 'm^3'

# compute storage change
if 'storage_change' not in list(reservoir_dynamics.variables):
    # avg_A = (reservoir_dynamics['area'].isel(date=slice(0, -1)) + reservoir_dynamics['area'].isel(date=slice(1, None)))/2
    # avg_A = reservoir_dynamics['area'].rolling(date=2).mean()
    # del_h = reservoir_dynamics['elevation'].diff(dim='date')
    # del_s = xr.DataArray(0.5 * avg_A * del_h * 1e6, name='storage_change')
    del_s = reservoir_dynamics['storage'].diff(dim='date')
    
    reservoir_dynamics = reservoir_dynamics.assign(storage_change=del_s)

reservoir_dynamics

<xarray.Dataset> Size: 3kB
Dimensions:         (date: 82)
Coordinates:
  * date            (date) datetime64[ns] 656B 2023-07-27 ... 2024-10-09
Data variables:
    elevation       (date) float64 656B 323.8 nan nan 324.3 ... 324.3 nan nan
    area            (date) float64 656B 588.7 nan nan 591.5 ... 591.2 nan nan
    storage         (date) float64 656B 1.579e+10 nan nan ... 1.596e+10 nan nan
    storage_change  (date) float64 656B nan nan nan nan nan ... nan nan nan nan

In [37]:
print(capacity)
(capacity_hv * (reservoir_dynamics*1e-6).hvplot.scatter(x='date', y='storage').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage (mil. m3)', ylabel='Storage', xlabel='Date'
))

36700.0


:Overlay
   .HLine.I   :HLine   [x,y]
   .Scatter.I :Scatter   [date]   (storage)

In [38]:
print(capacity)
(capacity_hv * (reservoir_dynamics*1e-6).hvplot.scatter(x='date', y='storage').opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME}. Storage (mil. m3)', ylabel='Storage', xlabel='Date'
))

36700.0


:Overlay
   .HLine.I   :HLine   [x,y]
   .Scatter.I :Scatter   [date]   (storage)

In [40]:
import hvplot.xarray

# Define the date range
date_range = (pd.to_datetime('2023-07-21'), pd.to_datetime('2024-10-30'))

# Plot storage change
storage_change_plot = (reservoir_dynamics['storage_change'] * 1e-6).hvplot.scatter(
    x='date', y='storage_change'
).opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME} - Storage Change', 
    ylabel='Storage Change (mil. m3)', 
    show_grid=True,
    xlim=date_range
) * hv.HLine(0).opts(color='gray', line_width=0.5)

# Plot area
area_plot = reservoir_dynamics.hvplot.scatter(
    x='date', y='area'
).opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME} - Area', 
    ylabel='Area (km2)', 
    show_grid=True,
    xlim=date_range
)

# Plot elevation
elevation_plot = reservoir_dynamics.hvplot.scatter(
    x='date', y='elevation'
).opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME} - Elevation', 
    ylabel='Elevation (m a.s.l)', 
    show_grid=True,
    xlim=date_range
)

# Plot area-elevation curve
area_elevation_curve = aec_df.hvplot.line(
    x='CumArea', y='Elevation'
).opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME} - Area-Elevation Curve', 
    xlabel='Area (km2)', 
    ylabel='Elevation (m a.s.l)', 
    show_grid=True
)

# Plot elevation values on the area-elevation curve
elevation_points = reservoir_dynamics.hvplot.scatter(
    x='area', y='elevation', color='red', by='date'
).opts(
    title=f'{RESERVOIR}: {RESERVOIR_NAME} - Elevation Points on AEC', 
    xlabel='Area (km2)', 
    ylabel='Elevation (m a.s.l)', 
    show_grid=True
)

# Combine the plots
time_series_plot = (storage_change_plot + area_plot + elevation_plot + (area_elevation_curve * elevation_points)).cols(1)
time_series_plot

:Layout
   .Overlay.I  :Overlay
      .Scatter.I :Scatter   [date]   (storage_change)
      .HLine.I   :HLine   [x,y]
   .Scatter.I  :Scatter   [date]   (area)
   .Scatter.II :Scatter   [date]   (elevation)
   .Overlay.II :Overlay
      .Curve.I     :Curve   [CumArea]   (Elevation)
      .NdOverlay.I :NdOverlay   [date]
         :Scatter   [area]   (elevation)